# System Dynamics and Equations of Motion

The first step we will take will be to derive the equations of motion of the system. We  will do this using the approach given in [Tuttle, 2014](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=1415&context=open_access_theses). We can do this using the Python package Sympy.

In [1]:
# Import everything we need from SymPy
import sympy as sym
import numpy as np
from sympy import Symbol, Matrix
from sympy import N as N_sympy
from sympy import simplify, sin, cos, init_printing
init_printing() # This function will make the outputs of SymPy look prettier and be easier to read

Now we are going to define the parameters of the system as constants:

In [2]:
# Constants of the system
m_c = 12.0    # Mass of the chassis in kg
ixx_c = 1.0   # Roll mass moment of ineria of the chassis
iyy_c = 0.8   # Pitch mass moment of interia of the chassis
izz_c = 0.52  # Yaw mass moment of interia of the chassis

m_w = 1.2       # Mass of either wheel in kg
iaa_w = 0.0634  # Axial mass moment of interia of the wheels
itt_w = 0.0327  # Transverse mass moment of interia of the wheels
r_w = 0.325     # Radius of either wheel in meters

m = m_c + 2*m_w        # The total mass of the system
ixx = ixx_c + 2*itt_w  # The total roll mass moment of inertia of the system
iyy = iyy_c            # The total pitch mass moment of inertia of the system
izz = izz_c + 2*itt_w  # The total yaw mass moment of inertia of the system

h = 0.2   # Distance from the CoM of the chassis to wheel axle in meters
a = 0.35  # Half the distance between the wheels in meters

r_station = 20.0  # The radius of the station in meters
v_station = -0.1  # The angular velocity of the station in rad/second

Now we make symbols and functions. These are elements of SymPy and can be thought of as exactly the same as symbols (variables) and functions from math.

In [3]:
# Create the system's generalized coordinates 
e_lat = Symbol('e_lat')          # The lateral error of the segbot. Positive is to the left of the target position.
phi = Symbol('phi')              # The yaw of the chassis of the segbot relative to the station. Positive is facing left.
phidot = Symbol('phidot')        # The yawing velocity of the segbot relative to the station. Positive is turning left.
v = Symbol('v')                  # The forward velocity of the segbot. Positive is moving forward.
theta = Symbol('theta')          # The pitch of the chassis of the segbot. Positive is tipped forwards.
thetadot = Symbol('thetadot')    # The pitching velocity of the segbot. Positive is tipping forward.
tau_l = Symbol('tau_l')          # The torque applied to the left wheel of the segbot. Positive moves the segbot forward.
tau_r = Symbol('tau_r')          # The torque applied to the right wheel of the segbot. Positive moves the segbot forward.


Now we apply the method given in [Tuttle, 2014](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=1415&context=open_access_theses).

In [4]:
M = Matrix([[m + 2 * iaa_w / r_w**2, 0, m_c * h * cos(theta)],
            [0, (ixx + m_c * h**2) * sin(theta)**2 + izz * cos(theta)**2 + (2 * iaa_w * a**2 / r_w**2) + 2 * m_w * a**2, 0],
            [m_c * h * cos(theta), 0, iyy * m_c * h**2]])
N = Matrix([[m_c * h * (phidot**2 + thetadot**2) * sin(theta)],
            [-2 * (ixx - izz + m * h**2) * cos(theta) * sin(theta) * phidot * thetadot - m_c * h * sin(theta) * v * phidot],
            [(ixx - izz + m_c * h**2) * cos(theta) * sin(theta) * phidot**2 + m_c * 9.81 * h * sin(theta)]])
R = Matrix([[1 / r_w, 1 / r_w],
            [-a / r_w, a / r_w],
            [-1, -1]])
f_partial = simplify(M.inv() * (N + R * Matrix([[tau_l], [tau_r]])))
f = Matrix([[v * sin(phi)],
            [f_partial]])
f = simplify(f)

$f$ is our system model such that
$$
\begin{bmatrix}
\dot{e_{lat}} \\
\dot{v} \\
\ddot{\phi} \\
\ddot{\theta}
\end{bmatrix} = f(e_{lat}, v, \phi, \dot{\phi}, \theta, \dot{\theta}, \tau_l, \tau_r)
$$

In [5]:
N_sympy(f, 3)  # This rounds floating point number to 3 places and then prints to the screen

⎡                                                  v⋅sin(φ)                    ↪
⎢                                                                              ↪
⎢                               ⎛ 2    2⎞               ⎛      2               ↪
⎢     -18.4⋅τₗ - 18.4⋅τᵣ - 14.4⋅⎝φ̇  + θ̇ ⎠⋅sin(θ) + 37.4⋅⎝0.48⋅φ̇ ⋅sin(2⋅θ) - τₗ ↪
⎢     ──────────────────────────────────────────────────────────────────────── ↪
⎢                                                     2                        ↪
⎢                                             89.9⋅cos (θ) - 93.5              ↪
⎢                                                                              ↪
⎢                           -1.06⋅φ̇⋅θ̇⋅sin(2⋅θ) - 2.4⋅φ̇⋅v⋅sin(θ) - 1.08⋅τₗ + 1. ↪
⎢                           ────────────────────────────────────────────────── ↪
⎢                                                     2                        ↪
⎢                                             0.96⋅sin (θ) + 1.03              ↪
⎢                     

Note that this system of equations is not in standard form — you will have to do this, as usual.

# System Linearization and State Space Form

## Place the System in Standard Form

Place the system in standard form by replacing each second order ODEs with two first order ODEs via the following substitution:
$$
\dot{\phi} = \omega_{\phi}
$$
$$
\dot{\theta} = \omega_{\theta}
$$

In [6]:
# Create the two new variables to place the system in standard form
(omega_phi, omega_theta) = sym.symbols('omega_phi, omega_theta')

# Add omega_theta's and omega_phi's definition as two new ODEs to the system
f = sym.Matrix([f[0],
                f[1],
                f[2],
                f[3],
                omega_phi,
                omega_theta])

# Make the change of variables dot{\phi} = \omega_{\phi} and dot{\theta} = \omega_{\theta} to place f in standard form
f = f.subs({phidot : omega_phi,
            thetadot : omega_theta})

sym.N(f, 3) # Display f in standard form

⎡                                                      v⋅sin(φ)                ↪
⎢                                                                              ↪
⎢                               ⎛  2         2⎞               ⎛       2        ↪
⎢     -18.4⋅τₗ - 18.4⋅τᵣ - 14.4⋅⎝ωᵩ  + ωₜₕₑₜₐ ⎠⋅sin(θ) + 37.4⋅⎝0.48⋅ωᵩ ⋅sin(2⋅ ↪
⎢     ──────────────────────────────────────────────────────────────────────── ↪
⎢                                                        2                     ↪
⎢                                                89.9⋅cos (θ) - 93.5           ↪
⎢                                                                              ↪
⎢                           -1.06⋅ωᵩ⋅ωₜₕₑₜₐ⋅sin(2⋅θ) - 2.4⋅ωᵩ⋅v⋅sin(θ) - 1.08⋅ ↪
⎢                           ────────────────────────────────────────────────── ↪
⎢                                                        2                     ↪
⎢                                                0.96⋅sin (θ) + 1.03           ↪
⎢                           

## Find an Equilibrium Point

We first chose a candidate equilibrium point, then we verify that $f(m_e, n_e) = 0$ to confirm our candidate equilibrium point is valid.

In [7]:
# Convert symbolic system to a numeric function
f_num = sym.lambdify([e_lat, v, omega_phi, omega_theta, phi, theta, tau_l, tau_r], f)

# Guess an equilibrium state of all 0s.
e_lat_e = 0.0 
v_e = -v_station
omega_phi_e = 0.0 
omega_theta_e = 0.0
phi_e = 0.0
theta_e = 0.0
tau_l_e = 0.0
tau_r_e = 0.0

# Test if our guess is an equilibrium position
f_num(e_lat_e, v_e, omega_phi_e, omega_theta_e, phi_e, theta_e, tau_l_e, tau_r_e)

array([[ 0.],
       [-0.],
       [ 0.],
       [-0.],
       [ 0.],
       [ 0.]])

## Linearize the System to get A and B

We now linearize the system about our equilibrium point to get the $A$ and $B$ matrices.

In [8]:
# Define the nonlinear state vector
m = [e_lat, v, omega_phi, omega_theta, phi, theta]

# Take the jacobian of f with respect to the nonlinear state vector
A_sym = f.jacobian(m)

# Convert the symbolic jacobian to a numeric representation
A_num = sym.lambdify([e_lat, v, omega_phi, omega_theta, phi, theta, tau_l, tau_r], A_sym)

# Evalulate the jacobian of f with respect to the nonlinear state vector at the equilibrium point
A = A_num(e_lat_e, v_e, omega_phi_e, omega_theta_e, phi_e, theta_e, tau_l_e, tau_r_e)

# Print A
sym.N(sym.Matrix(A),3)

⎡0  0   0    0   0.1     0   ⎤
⎢                            ⎥
⎢0  0   0    0    0   -245.0 ⎥
⎢                            ⎥
⎢0  0   0    0    0      0   ⎥
⎢                            ⎥
⎢0  0   0    0    0   1.59e+3⎥
⎢                            ⎥
⎢0  0  1.0   0    0      0   ⎥
⎢                            ⎥
⎣0  0   0   1.0   0      0   ⎦

In [9]:
# Define the nonlinear input vector
n = [tau_l, tau_r]

# Take the jacobian of f with respect to the nonlinear input vector
B_sym = f.jacobian(n)

# Convert the symbolic jacobian to a numeric representation
B_num = sym.lambdify([e_lat, v, omega_phi, omega_theta, phi, theta, tau_l, tau_r], B_sym)

# Evalulate the jacobian of f with respect to the nonlinear input vector at the equilibrium point
B = B_num(e_lat_e, v_e, omega_phi_e, omega_theta_e, phi_e, theta_e, tau_l_e, tau_r_e)

# Print B
sym.N(sym.Matrix(B),3)

⎡  0      0  ⎤
⎢            ⎥
⎢15.5   15.5 ⎥
⎢            ⎥
⎢-1.05  1.05 ⎥
⎢            ⎥
⎢-99.7  -99.7⎥
⎢            ⎥
⎢  0      0  ⎥
⎢            ⎥
⎣  0      0  ⎦

## Check the controllability of the linearized system

We build the controllability matrix, $W$, and ensure its rank rank is equal to the number of states.

In [10]:
# Check controllability
W = np.hstack((B, A@B, A@A@B, A@A@A@B, A@A@A@A@B, A@A@A@A@A@B))
assert(np.linalg.matrix_rank(W) == len(m))   

# Gain Selection and Controller Design

Given our linear model of the system, we choose $Q$ and $R$ matrices and solve the Algebraic Riccati Equation. To this solution, we can apply the Linear Quadratic Regulator to solve for a linear optimal gain matrix, $K$.

In [11]:
# Get a reasonable Q and R for LQR
Q = np.eye(6)
R = np.eye(2)
Q[0,0] = 1. / (1.25**2)
Q[1,1] = 1. / (1.0**2)
Q[2,2] = 1. / (3.0**2)
Q[3,3] = 1. / (0.5**2)
Q[4,4] = 1. / (0.5**2)
Q[5,5] = 1. / (0.05**2)
R[0,0] = 1. / (0.5**2)
R[1,1] = 1. / (0.5**2)

In [12]:
# Solve the Algebraic Riccati Equation to get P, then use P, R, and B to get the optimal K
from scipy.linalg import solve_continuous_are as are_soln
P = are_soln(A, B, Q, R)
K = np.linalg.inv(R) @ B.T @ P
sym.N(sym.Matrix(K),3) # Print the control gains

⎡-0.283  -0.354  -0.848  -0.886  -0.74  -18.8⎤
⎢                                            ⎥
⎣0.283   -0.354  0.848   -0.886  0.74   -18.8⎦

It's interesting that each entry in the first row and the second row ($\tau_l$ and $\tau_r$, respectively) have the same magnitude, but not always the same sign. How could you explain this via intuition of the system and its dynamics? 

In [13]:
# Check to ensure the closed-loop system is stable
F = A-B@K
val, vec = np.linalg.eig(F)
sym.N(sym.Matrix(val),3) # Print the eigenvalues

⎡    -152.0     ⎤
⎢               ⎥
⎢-0.87 + 0.853⋅ⅈ⎥
⎢               ⎥
⎢-0.87 - 0.853⋅ⅈ⎥
⎢               ⎥
⎢     -0.04     ⎥
⎢               ⎥
⎢     -14.0     ⎥
⎢               ⎥
⎣    -0.104     ⎦

In [14]:
# Create the Controller class
class Controller():
    def __init__(self):
        """
        Initializes the controller. You should initialize any member variables that
        the controller will use to their starting values. This function is called
        when a new instance of the Controller class is created.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # Set control gains
        self.K = K

        # Set the equilibrium point
        self.e_lat_e = e_lat_e
        self.v_e = v_e
        self.omega_phi_e = omega_phi_e
        self.omega_theta_e = omega_theta_e
        self.phi_e = phi_e
        self.theta_e = theta_e
        self.tau_l_e = tau_l_e
        self.tau_r_e = tau_r_e
    
    def reset(self):
        """
        Resets the controller to an initial state. You should reset any variables
        that track states, times, etc. in this function. This function will be 
        called just before the simulation starts running and any time the simulation
        is reset.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # The reset function is not used
        pass 

    
    def run(self, **kwargs):
        """
        Runs the controller. Data is passed from the simulation environment to the
        controller via kwargs, the controller calculates the inputs to the system,
        and then returns them via a return statement
    
        Parameters
        ----------
        **kwargs : Dictionary
            A dictionary of data passed from the simulation environment to the 
            controller. Specific data is extracted from kwargs via the statement:
            kwargs["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys
            of kwargs for this project are as follows:

            kwargs["e_lat"] : Float
                The current lateral position error of the segbot in meters.
            
            kwargs["v"] : Float
                The current forward velocity of the segbot in meters/second.
            
            kwargs["phi"] : Float
                The current heading angle of the chassis in radians.
            
            kwargs["phidot"] : Float
                The current heading  angular rate of the chassis in radians/second.
            
            kwargs["theta"] : Float
                The current pitch angle of the chassis in radians.
            
            kwargs["thetadot"] : Float
                The current pitch angular rate of the chassis in radians/second.
    
        Returns
        -------
        inputs : List with length equal to the number of inputs
            This is a list of the inputs calculated by the controller. In this
            project, there are two inputs, the torque applied to the left wheel
            and the torque applied to the right wheel. Therefore, inputs has the 
            form:
            inputs = [left_torque, right_torque]
        
        """
        # Get the inputs using the control law u=-Kx
        x = np.array([kwargs["e_lat"] - self.e_lat_e,
                      kwargs["v"] - self.v_e,
                      kwargs["phidot"] - self.omega_phi_e,
                      kwargs["thetadot"] - self.omega_theta_e,
                      kwargs["phi"] - self.phi_e,
                      kwargs["theta"] - self.theta_e])
        u = -K@x
        n = u + np.array([self.tau_l_e,
                          self.tau_r_e])

        # Format and return the inputs
        inputs = n.tolist()
        return inputs

In [15]:
# Create an instance of our Controller class
controller = Controller()

# Running the Simulation

Before we run the simulation, we must first import the simulation module and then create an instance of the simulator. This is done below:

In [16]:
# Import the segbot project. This module is used to simulate, render, and plot a segbot dynamic system
import ae353_segbot

pybullet build time: Jan 20 2023 17:42:59


There are four main parameters to the Segbot_sim class initialization function:
#### Parameters

> **use_keyboard** : *bool, optional*  
> A boolean flag that indicates whether the simulation will allow the use of keyboard interactivity. The default is True.
> 
> **visualization** : *bool, optional*  
> A boolean flag that indicates whether the simulation will be  visualized in meshcat. The default is True.
> 
> **animation** : *bool, optional*  
> A boolean flag that indicates whether animated plots are created in real time. The default is True.
> 
> **bumpy**: *bool, optional*  
> A boolean flag that indicates whether the station will be built with bumps on the driving surface or not.

In [20]:
# Create an instance of the segbot simulator
sim = ae353_segbot.Segbot_sim(use_keyboard=True,
                              visualization=True,
                              animation=False,
                              bumpy=True)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
argv[0]=
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
space

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


Now we are ready to run the simulation and collect data. To do this we call the ``sim.run()`` function. This function takes an instance of the ``Controller`` class as an argument and returns ``data``. The parameters and return values of ``sim.run()`` are shown below.

#### Parameters

> **controller** : *member of Controller class*  
> Your controller that will generated inputs to the system.
> 
> **max_time** : *Float or None, optional*  
> The total amount of time the simulation is allowed to run. If set to None, the simulation will run until "ESC" is pressed on the keyboard. If the keyboard is disabled, and max_time is set to None, the simulation will automatically terminate after 10 seconds of simulated time. The default value is None.
> 
> **initial_e_lat** : *Float, optional*  
> The initial lateral error in meters. Changes the target lateral error, does not change the lateral position of the segbot. The default value is 0.0.
> 
> **station_velocity** : *Float, optional*  
> The constant rotational speed of the station in rad/s. The default value is -0.1.

#### Returns

> **data** : *Dictionary of Lists*  
> A dictionary containing all relevant data generated during the simulation. Specific data is extracted via the statement: data["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys of data for this project are as follows:
>  
> **data["lat_pos"]** : *List of Floats*  
> A list of the lateral position of the segbot in meters at each time stamp during the simulation.
>  
> **data["tag_lat"]** : *List of Floats*  
> A list of the taget lateral position of the segbot in meters at each time stamp during the simulation.  
>  
> **data["e_lat"]** : *List of Floats*  
> A list of the lateral position error of the segbot in meters at each time stamp during the simulation.
>  
> **data["v"]** : *List of Floats*
> test
>  
> A list of the forward velocity of the segbot in meters/second at each time stamp during the simulation.
> 
> **data["phi"]** : *List of Floats*  
> A list of the heading angle of the chassis in radians at each time stamp during the simulation.
> 
> **data["phidot"]** : *List of Floats*  
> A list of the heading  angular rate of the chassis in radians/second at each time stamp during the simulation.
> 
> **data["theta"]** : *List of Floats*  
> A list of the pitch angle of the chassis in radians at each time stamp during the simulation.
> 
> **data["thetadot"]** : *List of Floats*  
> A list of the pitch angular rate of the chassis in radians/second at each time stamp during the simulation.
> 
> **data["tau_l"]** : *List of Floats*  
> A list of the torque applied to the left wheel of the segbot in Newton-meters at each time stamp during the simulation.
> 
> **data["tau_r"]** : *List of Floats*  
> A list of the torque applied to the right wheel of the segbot in Newton-meters at each time stamp during the simulation.
> 
> **data["time"]** : *List of Floats*  
> A list of the time stamps in seconds.

#### If enabled you can use the keyboard to interact with the simulation:  
> press **ENTER** to start the simulation  
> press **BACKSPACE** to reset the simulation  
> press **SPACE** to pause the simulation  
> press **ESC** to end the simulation  

#### To control the target lateral position:
> press **j** to move the target to the left  
> press **l** to move the target to the right  

#### If the manual controller is being used:
> press **w** to apply small forward torque to both wheels  
> press **SHIFT+w** to apply large forward torque to both wheels  
> press **s** to apply small backward torque to both wheels  
> press **SHIFT+s** to apply large backward torque to both wheels  
> press **d** to apply small forward torque to only the left wheel  
> press **SHIFT+d** to apply large forward torque to only the left wheel  
> press **a** to apply small backward torque to only the right wheel  
> press **SHIFT+a** to apply large backward torque to only the right wheel  

In [ ]:
# Run the simulation and collect the simulation data
data = sim.run(controller,
               max_time=None,
               initial_e_lat = 0.0,
               station_velocity = v_station)

PRESS ENTER TO START SIMULATION.
PRESS ESC TO QUIT.
PRESS SPACE TO PAUSE/RESUME SIMULATION.
PRESS BACKSPACE TO RESET SIMULATION.
CONTINUING...


# Post-processing the simulation data

After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib.

In [ ]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

Now we can make whichever plots we see fit to make sense of the simulation data. Here we plot the pitch versus time as well as the applied torques versus time.

**THIS IS LEFT INTENTIONALLY AS INSUFFICIENT TO PROVE YOUR CONTROLLER CAN STEER THE SEGBOT.**

In [ ]:
# Create a figure that has two plots in it
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=True)

# Create a plot of the pitch angle versus time on the first plot
axes[0].plot(data['time'], 180.*np.array(data['theta'])/np.pi, c='k', lw=2.5)
axes[0].set_ylabel("Pitch [deg]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].set_title("Pitch vs. Time", fontsize=14)
axes[0].grid()

# Create a plot of the torque versus time on the second plot
axes[1].plot(data['time'], data['tau_l'], label="Left Torque", c='r', lw=2.5)
axes[1].plot(data['time'], data['tau_r'], label="Right Torque", c='b', lw=2.5)
axes[1].set_xlabel("Time [s]", fontsize=12)
axes[1].set_ylabel("Torque [Nm]", fontsize=12)
axes[1].tick_params(axis='y', labelsize=12)
axes[1].legend(fontsize=12, shadow=True)
axes[1].set_title("Torque vs. Time", fontsize=14)
axes[1].grid()

# Show the figure
fig.tight_layout(pad=2.0)
plt.show()